In [1]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datestart='2025-01-01'
dateend='2024--01'
datefilter=''

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date,fav_percent,dog_percent):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date >= '{datestart}'",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[(data['fav_percent']<fav_percent)&(data['dog_percent']>dog_percent)]
    data=data[data['Sex']==sex]
    
    
    if len(data)>0:
        data['Profit'] = data.apply(lambda row: -stake if row['Fav_Win'] else (stake * row['Dog_Odds']) - stake, axis=1)
    #    print(len(data[data['Winner']!=data['Fav']])/len(data),len(data),f"${data['Profit'].sum()}")
    return data


def hist_data_fav_elo(surface, sex, odds_low,odds_high,date,fav_elo,dog_elo):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date >= '{datestart}'",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    #data=data[(data['Elo_Fav_Elo']<fav_percent)&(data['Elo_Dog_Elo']>dog_percent)]
    data=data[(data['Elo_Dog_Elo']>=dog_elo)&(data['Elo_Fav_Elo']>=fav_elo)]
    data=data[data['Sex']==sex]
    
    if len(data)>0:
        data['Profit'] = data.apply(lambda row: -stake if not row['Fav_Win'] else (stake * row['Fav_Odds']) - stake, axis=1)
        print(len(data[data['Winner']==data['Fav']])/len(data),len(data),f"${data['Profit'].sum()}")
    return data

def hist_data_dog_elo(surface, sex, odds_low,odds_high,date,fav_percent,dog_elo):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date >= '{datestart}'",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    #data=data[(data['Elo_Fav_Elo']<fav_percent)&(data['Elo_Dog_Elo']>dog_percent)]
    data=data[(data['Elo_Dog_Elo'])>dog_elo]
    data=data[data['Sex']==sex]
    
    
    if len(data)>0:
        data['Profit'] = data.apply(lambda row: -stake if row['Fav_Win'] else (stake * row['Dog_Odds']) - stake, axis=1)
        print(len(data[data['Winner']!=data['Fav']])/len(data),len(data),f"${data['Profit'].sum()}")
    return data

In [2]:
df=hist_data_dog('Clay','Mens',1,2,datefilter,1,0)
filtered1=((df['dog_percent']>0)&(df['fav_percent']<0.3)&(df['Fav_Odds']>=1.5)&(df['Fav_Odds']<=1.6)) #mens clay 54% odds 1.5, 1.7
filtered2=((df['dog_percent']>0)&(df['fav_percent']<0.3)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #mens clay 54% odds 1.5, 1.7
filtered3=((df['dog_percent']>0)&(df['fav_percent']<10)&(df['Dog_Top100']>0.6)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #mens clay 60% odds 1.7
filtered=df[filtered1|filtered2|filtered3]
if len(filtered)>0:
    print(len(filtered[filtered['Winner']!=filtered['Fav']])/len(filtered),len(filtered),f"${filtered['Profit'].sum()}")


0.3333333333333333 24 $-742.0


In [3]:
df=hist_data_dog('Clay','Womens',1,2,datefilter,1,0)
filtered=df[(df['dog_percent']>0.2)&(df['fav_percent']<1)&(df['Fav_Odds']>=1.5)&(df['Fav_Odds']<=1.6)] #womens clay 52% odds 1.5
if len(filtered)>0:
    print(len(filtered[filtered['Winner']!=filtered['Fav']])/len(filtered),len(filtered),f"${filtered['Profit'].sum()}")

0.4782608695652174 23 $334.0


In [4]:
df=hist_data_dog('Hard','Womens',1,2,datefilter,1,0)
filtered1=((df['dog_percent']>0.5)&(df['fav_percent']<1)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #womens clay 53% odds 1.7
filtered2=((df['dog_percent']>0.5)&(df['fav_percent']<0.6)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #womens clay 59% odds 1.7
filtered3=((df['dog_percent']>0.5)&(df['fav_percent']<5)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #womens clay 59% odds 1.7
filtered5=((df['dog_percent']>0)&(df['fav_percent']<0.5)&(df['Dog_Games']>10)&(df['Fav_Games']>10)&(df['Fav_Odds']>=1.5)&(df['Fav_Odds']<=1.9)) #1.5 - 1.9
filtered6=((df['dog_percent']>0.5)&(df['fav_percent']<5)&(df['Dog_Games']>10)&(df['Fav_Games']>10)&(df['Fav_Odds']>=1.6)&(df['Fav_Odds']<=1.9))
filtered7=((df['Dog_Top100']>0.4)&(df['fav_percent']<5)&(df['Dog_Games']>10)&(df['Fav_Games']>10)&(df['Fav_Odds']>=1.6)&(df['Fav_Odds']<=1.9)) #1.6 - 1.9
filtered8=((df['Fav_Top100']<0.5)&(df['fav_percent']<5)&(df['Dog_Games']>10)&(df['Fav_Games']>10)&(df['Fav_Odds']>=1.5)&(df['Fav_Odds']<=1.9)) #1.5 - 1.9
filtered21=((df['dog_percent']>0.5)&(df['fav_percent']<1)&(df['dog_rank']<=100)&(df['Fav_Odds']>=1.6)&(df['Fav_Odds']<=1.8)) #hard womens odds 1.6,1.7
filtered22=((df['dog_percent']>0)&(df['fav_percent']<1)&(df['dog_rank']<=100)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #hard womens odds 1.7
filtered24=((df['dog_percent']>0.5)&(df['fav_percent']<5)&(df['dog_rank']<=100)&(df['Fav_Odds']>=1.6)&(df['Fav_Odds']<=1.8)) #hard womens odds 1.6, 1.7
filtered25=((df['dog_percent']>0.5)&(df['fav_percent']<5)&(df['dog_rank']<=200)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #hard womens odds 1.7
filtered26=((df['dog_percent']>0)&(df['fav_percent']<0.8)&(df['dog_rank']<=100)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #hard womens odds 1.7
filtered=df[filtered1|filtered2|filtered3|filtered5|filtered6|filtered7|filtered8|filtered21|filtered22|filtered24|filtered25|filtered26]
if len(filtered)>0:
    print(len(filtered[filtered['Winner']!=filtered['Fav']])/len(filtered),len(filtered),f"${filtered['Profit'].sum()}")

0.47843137254901963 255 $512.0


In [5]:
df=hist_data_dog('Hard','Mens',1,2,datefilter,1,0)
filtered1=((df['Fav_Top100']<df['Dog_Top100'])&(df['dog_percent']>0.5)&(df['Fav_Odds']>=1.8)&(df['Fav_Odds']<=1.9)) #1.8, 58%
filtered2=((df['dog_percent']>0.5)&(df['fav_percent']<0.6)&(df['Dog_Games']>10)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #1.7,1.8 55%
filtered3=((df['dog_percent']>0.5)&(df['fav_percent']<0.5)&(df['Dog_Games']>10)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #1.7, 60%
filtered4=((df['dog_percent']>0)&(df['fav_percent']<0.4)&(df['Dog_Games']>10)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #1.7, 55%
filtered5=((df['dog_percent']>0)&(df['fav_percent']<0.5)&(df['Dog_Games']>10)&(df['Fav_Games']>10)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #1.7, 55%
filtered6=((df['dog_percent']>0.3)&(df['fav_percent']<0.5)&(df['Dog_Games']>10)&(df['Fav_Games']>10)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8)) #1.7, 56%
filtered11=((df['dog_percent']>0.5)&(df['fav_percent']<0.6)&(df['Fav_Odds']>=1.5)&(df['Fav_Odds']<=1.6)) #mens hard 1.5, 1.8
filtered31=((df['dog_percent']>0.5)&(df['fav_percent']<0.5)&(df['Fav_Odds']>=1.5)&(df['Fav_Odds']<=1.6)) #mens hard 1.5, 1.7, 1.8
filtered41=((df['dog_percent']>0.5)&(df['fav_percent']<0.5)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8))
filtered61=((df['dog_percent']>0.5)&(df['fav_percent']<10)&(df['Fav_Odds']>=1.8)&(df['Fav_Odds']<=1.9)) #mens hard 58% odds 1.8    
filtered01=((df['dog_percent']>0.5)&(df['fav_percent']<0.6)&(df['Fav_Odds']>=1.5)&(df['Fav_Odds']<=1.6)) #mens hard 1.5, 1.8
filtered03=((df['dog_percent']>0.5)&(df['fav_percent']<0.5)&(df['Fav_Odds']>=1.5)&(df['Fav_Odds']<=1.6)) #mens hard 1.5, 1.7, 1.8
filtered04=((df['dog_percent']>0.5)&(df['fav_percent']<0.5)&(df['Fav_Odds']>=1.7)&(df['Fav_Odds']<=1.8))
filtered06=((df['dog_percent']>0.5)&(df['fav_percent']<10)&(df['Fav_Odds']>=1.8)&(df['Fav_Odds']<=1.9)) #mens hard 58% odds 1.8
filtered=df[filtered1|filtered2|filtered3|filtered4|filtered5|filtered6|filtered11|filtered31|filtered41|filtered61|filtered01|filtered03|filtered04|filtered06]
if len(filtered)>0:
    print(len(filtered[filtered['Winner']!=filtered['Fav']])/len(filtered),len(filtered),f"${filtered['Profit'].sum()}")

0.44680851063829785 188 $-1307.0
